In [314]:
import pandas as pd
import numpy as np
df = pd.read_csv('vk.csv')
df.head()

,Unnamed: 0,question,answer
0,0,кредитная карта моментум. ею можно расплачива...,"вы можете, использовать кредитую карту visa/..."
1,1,"на дебетную карту пришло сообщение о ""неразреш...",по данному вопросу вы можете обратиться в сл...
2,2,вопрос. почему не приходят смс о пополнении ка...,по данному вопросу вы можете обратиться в сл...
3,3,"есть зарплатная карта сб, пользуюсь сбол. кро...",по данному вопросу вы можете обратиться в сл...
4,4,"здраствуйте.мне пришло смс ,что одобрен предво...",потребительский кредит может быть выдан при ...


In [315]:
import logging
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
from gensim import corpora


In [316]:
documents = df['question'].tolist()

In [317]:
import re
documents = [str(x)
             .replace("тк","")
             .replace("сбербанк","банк")
             .replace("сбер","банк")
             .replace("сбербанка","банк")
             .replace("банка","банк")
             for x in documents]
documents = [re.sub(r'[^а-яА-Я ]+', '',str(x))
             for x in documents]
print(documents[1])

на дебетную карту пришло сообщение о неразрешенной задолженности никакой задолженности быть не может тк карта дебетная на сколько я понял это банковский косяк и с меня банк захочет взять процент как мне вернуть свои деньги


In [318]:
import nltk
from nltk.corpus import brown
stop_words= nltk.corpus.stopwords.words('russian')
stop_word="пожалуйста здоавствуйте"
for i in stop_words:
        stop_word=  stop_word+" "+i
stoplist = stop_word

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in documents]


In [319]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [320]:
texts = [[token for token in text if frequency[token] > 2] for text in texts]

In [321]:
#lemmatize
import time
start = time.time()
import pymystem3
mystem = pymystem3.Mystem()
texts_lemmatized = [[mystem.lemmatize(token)[0] for token in text] for text in texts]
stop = time.time()

In [322]:
print("Время, затраченное lemmatize на обработку слов "+str((stop - start)))

Время, затраченное lemmatize на обработку слов 133.1373062133789


In [323]:
texts = texts_lemmatized
print(texts[0:3])

[['кредитный', 'карта', 'моментум', 'она', 'расплачиваться', 'зарубежный', 'интернет', 'магазин'], ['дебетный', 'карта', 'приходить', 'сообщение', 'задолженность', 'никакой', 'задолженность', 'тк', 'карта', 'дебетный', 'сколько', 'понимать', 'банковский', 'косяк', 'банк', 'захотеть', 'взять', 'процент', 'вернуть', 'свой', 'деньги'], ['вопрос', 'почему', 'приходить', 'смс', 'пополнение', 'карта', 'ради', 'подключный', 'мобильный', 'банк']]


In [324]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)


2017-10-12 16:38:48,184 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-10-12 16:38:48,689 : INFO : adding document #10000 to Dictionary(5582 unique tokens: ['кредитный', 'карта', 'моментум', 'она', 'расплачиваться']...)
2017-10-12 16:38:49,188 : INFO : adding document #20000 to Dictionary(6449 unique tokens: ['кредитный', 'карта', 'моментум', 'она', 'расплачиваться']...)
2017-10-12 16:38:49,771 : INFO : adding document #30000 to Dictionary(6721 unique tokens: ['кредитный', 'карта', 'моментум', 'она', 'расплачиваться']...)
2017-10-12 16:38:50,221 : INFO : adding document #40000 to Dictionary(6800 unique tokens: ['кредитный', 'карта', 'моментум', 'она', 'расплачиваться']...)
2017-10-12 16:38:50,230 : INFO : built Dictionary(6800 unique tokens: ['кредитный', 'карта', 'моментум', 'она', 'расплачиваться']...) from 40117 documents (total 679336 corpus positions)
2017-10-12 16:38:50,238 : INFO : saving Dictionary object under /var/folders/c1/98z6jlhd21z3y8kq9zxhyxsw0000gn

Dictionary(6800 unique tokens: ['кредитный', 'карта', 'моментум', 'она', 'расплачиваться']...)


In [325]:
new_doc = "чат бот"
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(4628, 1)]

In [326]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus) 

2017-10-12 16:38:51,623 : INFO : storing corpus in Matrix Market format to /var/folders/c1/98z6jlhd21z3y8kq9zxhyxsw0000gn/T/deerwester.mm
2017-10-12 16:38:51,625 : INFO : saving sparse matrix to /var/folders/c1/98z6jlhd21z3y8kq9zxhyxsw0000gn/T/deerwester.mm
2017-10-12 16:38:51,627 : INFO : PROGRESS: saving document #0
2017-10-12 16:38:51,669 : INFO : PROGRESS: saving document #1000
2017-10-12 16:38:51,701 : INFO : PROGRESS: saving document #2000
2017-10-12 16:38:51,735 : INFO : PROGRESS: saving document #3000
2017-10-12 16:38:51,773 : INFO : PROGRESS: saving document #4000
2017-10-12 16:38:51,811 : INFO : PROGRESS: saving document #5000
2017-10-12 16:38:51,848 : INFO : PROGRESS: saving document #6000
2017-10-12 16:38:51,884 : INFO : PROGRESS: saving document #7000
2017-10-12 16:38:51,923 : INFO : PROGRESS: saving document #8000
2017-10-12 16:38:51,963 : INFO : PROGRESS: saving document #9000
2017-10-12 16:38:52,000 : INFO : PROGRESS: saving document #10000
2017-10-12 16:38:52,035 : INF

In [327]:
from gensim import corpora, models, similarities
if os.path.isfile(os.path.join(TEMP_FOLDER, 'deerwester.dict')):
    dictionary = corpora.Dictionary.load(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
    corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))
else:
    print("generate data")

2017-10-12 16:38:53,117 : INFO : loading Dictionary object from /var/folders/c1/98z6jlhd21z3y8kq9zxhyxsw0000gn/T/deerwester.dict
2017-10-12 16:38:53,125 : INFO : loaded /var/folders/c1/98z6jlhd21z3y8kq9zxhyxsw0000gn/T/deerwester.dict
2017-10-12 16:38:53,142 : INFO : loaded corpus index from /var/folders/c1/98z6jlhd21z3y8kq9zxhyxsw0000gn/T/deerwester.mm.index
2017-10-12 16:38:53,143 : INFO : initializing corpus reader from /var/folders/c1/98z6jlhd21z3y8kq9zxhyxsw0000gn/T/deerwester.mm
2017-10-12 16:38:53,145 : INFO : accepted corpus with 40117 documents, 6800 features, 582248 non-zero entries


In [330]:
print(dictionary[0])
print(dictionary[1])
print(dictionary[2])

кредитный
карта
моментум


In [331]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=150, update_every=1, chunksize=10000, passes=5)

2017-10-12 16:44:10,220 : INFO : using symmetric alpha at 0.006666666666666667
2017-10-12 16:44:10,222 : INFO : using symmetric eta at 0.00014705882352941175
2017-10-12 16:44:10,226 : INFO : using serial LDA version on this node
2017-10-12 16:44:18,123 : INFO : running online (multi-pass) LDA training, 150 topics, 5 passes over the supplied corpus of 40117 documents, updating model once every 10000 documents, evaluating perplexity every 40117 documents, iterating 50x with a convergence threshold of 0.001000
2017-10-12 16:44:18,832 : INFO : PROGRESS: pass 0, at document #10000/40117
2017-10-12 16:44:44,359 : INFO : merging changes from 10000 documents into a model of 40117 documents
2017-10-12 16:44:44,682 : INFO : topic #92 (0.007): 0.040*"карта" + 0.029*"мочь" + 0.027*"деньги" + 0.019*"банк" + 0.013*"сказать" + 0.012*"кредитный" + 0.010*"делать" + 0.009*"год" + 0.009*"который" + 0.009*"получать"
2017-10-12 16:44:44,684 : INFO : topic #10 (0.007): 0.033*"деньги" + 0.027*"банк" + 0.021*

2017-10-12 16:46:14,620 : INFO : topic #142 (0.007): 0.202*"счт" + 0.040*"орытие" + 0.034*"деньги" + 0.031*"карта" + 0.030*"дедушка" + 0.028*"трх" + 0.026*"полагать" + 0.021*"работать" + 0.020*"банк" + 0.016*"класть"
2017-10-12 16:46:14,622 : INFO : topic #66 (0.007): 0.138*"приложение" + 0.100*"банк" + 0.085*"онлайн" + 0.056*"мочь" + 0.048*"входить" + 0.042*"вводить" + 0.030*"оно" + 0.021*"мобильный" + 0.019*"запрашиваться" + 0.019*"логин"
2017-10-12 16:46:14,630 : INFO : topic diff=0.791205, rho=0.407851
2017-10-12 16:46:15,321 : INFO : PROGRESS: pass 1, at document #20000/40117
2017-10-12 16:46:35,002 : INFO : merging changes from 10000 documents into a model of 40117 documents
2017-10-12 16:46:35,302 : INFO : topic #16 (0.007): 0.080*"пример" + 0.053*"пенсионер" + 0.043*"карта" + 0.042*"банк" + 0.041*"граф" + 0.033*"момент" + 0.031*"достоверный" + 0.028*"причем" + 0.020*"тд" + 0.019*"подсказывать"
2017-10-12 16:46:35,304 : INFO : topic #23 (0.007): 0.059*"банк" + 0.040*"деньги" + 0

2017-10-12 16:48:00,865 : INFO : topic #70 (0.007): 0.114*"лицевой" + 0.083*"обслуживание" + 0.064*"карта" + 0.061*"деньги" + 0.043*"общий" + 0.038*"понятно" + 0.037*"стоить" + 0.037*"сколько" + 0.033*"срочно" + 0.024*"переводиться"
2017-10-12 16:48:00,868 : INFO : topic #14 (0.007): 0.094*"приходить" + 0.083*"карта" + 0.058*"смс" + 0.056*"также" + 0.054*"должный" + 0.046*"конец" + 0.046*"кредитный" + 0.045*"сумма" + 0.042*"сегодня" + 0.040*"смска"
2017-10-12 16:48:00,877 : INFO : topic diff=1.000581, rho=0.377649
2017-10-12 16:48:01,390 : INFO : PROGRESS: pass 2, at document #30000/40117
2017-10-12 16:48:25,042 : INFO : merging changes from 10000 documents into a model of 40117 documents
2017-10-12 16:48:25,256 : INFO : topic #100 (0.007): 0.080*"активировать" + 0.060*"хороший" + 0.044*"тема" + 0.043*"помощь" + 0.032*"всякий" + 0.030*"замечать" + 0.023*"специально" + 0.022*"ненужный" + 0.021*"планироваться" + 0.021*"клиент"
2017-10-12 16:48:25,260 : INFO : topic #114 (0.007): 0.064*"о

2017-10-12 16:49:50,792 : INFO : topic #149 (0.007): 0.286*"перевод" + 0.150*"карта" + 0.143*"банк" + 0.041*"деньги" + 0.036*"сделать" + 0.030*"другой" + 0.022*"клиент" + 0.020*"онлайн" + 0.019*"комиссия" + 0.016*"мой"
2017-10-12 16:49:50,794 : INFO : topic #64 (0.007): 0.248*"оплачивать" + 0.063*"квитанция" + 0.063*"доходить" + 0.060*"банк" + 0.045*"онлайн" + 0.042*"пункт" + 0.038*"искать" + 0.027*"свой" + 0.026*"втб" + 0.025*"внесение"
2017-10-12 16:49:50,796 : INFO : topic #20 (0.007): 0.117*"проводить" + 0.084*"попросить" + 0.055*"отправитель" + 0.054*"банк" + 0.041*"изза" + 0.038*"вывод" + 0.034*"вконтакте" + 0.033*"оператор" + 0.033*"неверный" + 0.025*"клиент"
2017-10-12 16:49:50,807 : INFO : topic diff=0.594139, rho=0.353295
2017-10-12 16:49:51,318 : INFO : PROGRESS: pass 3, at document #40000/40117
2017-10-12 16:50:10,605 : INFO : merging changes from 10000 documents into a model of 40117 documents
2017-10-12 16:50:10,761 : INFO : topic #4 (0.007): 0.195*"имя" + 0.054*"самостоя

2017-10-12 16:51:31,707 : INFO : topic #38 (0.007): 0.289*"месяц" + 0.113*"каждый" + 0.062*"годовой" + 0.046*"обслуживание" + 0.041*"сниматься" + 0.036*"мало" + 0.028*"такой" + 0.028*"начисление" + 0.026*"рубль" + 0.023*"получать"
2017-10-12 16:51:31,709 : INFO : topic #119 (0.007): 0.308*"оформлять" + 0.214*"карта" + 0.175*"кредитный" + 0.067*"банк" + 0.061*"мочь" + 0.030*"вопрос" + 0.017*"сказать" + 0.014*"получать" + 0.011*"возможно" + 0.010*"хотеть"
2017-10-12 16:51:31,711 : INFO : topic #64 (0.007): 0.309*"оплачивать" + 0.064*"банк" + 0.062*"квитанция" + 0.061*"доходить" + 0.047*"онлайн" + 0.038*"пункт" + 0.037*"искать" + 0.026*"втб" + 0.024*"изменение" + 0.024*"свой"
2017-10-12 16:51:31,713 : INFO : topic #46 (0.007): 0.135*"комиссия" + 0.083*"банк" + 0.069*"карта" + 0.068*"рубль" + 0.063*"наличные" + 0.047*"снятие" + 0.040*"курс" + 0.040*"валюта" + 0.040*"доллар" + 0.029*"банкомат"
2017-10-12 16:51:31,721 : INFO : topic diff=0.305429, rho=0.333117
2017-10-12 16:51:32,140 : INFO 

In [332]:
lda.print_topics(num_topics=250, num_words=10)

2017-10-12 16:51:32,530 : INFO : topic #0 (0.007): 0.189*"личный" + 0.158*"пароль" + 0.149*"кабинет" + 0.092*"банк" + 0.073*"зайти" + 0.067*"онлайн" + 0.050*"мочь" + 0.022*"заходить" + 0.021*"писать" + 0.019*"логин"
2017-10-12 16:51:32,533 : INFO : topic #1 (0.007): 0.144*"адрес" + 0.115*"заполнять" + 0.085*"проверка" + 0.080*"мир" + 0.064*"верить" + 0.060*"электронный" + 0.053*"ульяновск" + 0.041*"почта" + 0.031*"анкета" + 0.028*"банк"
2017-10-12 16:51:32,534 : INFO : topic #2 (0.007): 0.110*"дата" + 0.103*"оператор" + 0.094*"хотеться" + 0.084*"утро" + 0.076*"добрый" + 0.049*"изменяться" + 0.043*"узнавать" + 0.033*"связь" + 0.028*"помнить" + 0.028*"верный"
2017-10-12 16:51:32,537 : INFO : topic #3 (0.007): 0.321*"один" + 0.117*"регистрация" + 0.114*"разный" + 0.079*"выпускать" + 0.077*"два" + 0.070*"банк" + 0.057*"другой" + 0.043*"виртуальный" + 0.032*"карта" + 0.018*"онлайн"
2017-10-12 16:51:32,539 : INFO : topic #4 (0.007): 0.237*"имя" + 0.087*"посчитать" + 0.048*"самостоятельно" + 

2017-10-12 16:51:32,604 : INFO : topic #36 (0.007): 0.226*"руб" + 0.101*"поддержка" + 0.055*"служба" + 0.050*"потеряться" + 0.044*"тот" + 0.043*"связанный" + 0.032*"украсть" + 0.026*"видимо" + 0.023*"вопрос" + 0.020*"банк"
2017-10-12 16:51:32,606 : INFO : topic #37 (0.007): 0.314*"операция" + 0.115*"чек" + 0.097*"карта" + 0.044*"банкомат" + 0.044*"деньги" + 0.037*"информирование" + 0.026*"яндекс" + 0.025*"выдавать" + 0.018*"перевести" + 0.015*"перевл"
2017-10-12 16:51:32,608 : INFO : topic #38 (0.007): 0.258*"месяц" + 0.101*"каждый" + 0.055*"годовой" + 0.051*"начисление" + 0.044*"такой" + 0.041*"обслуживание" + 0.037*"сниматься" + 0.034*"зарание" + 0.032*"апсс" + 0.032*"мало"
2017-10-12 16:51:32,610 : INFO : topic #39 (0.007): 0.146*"становиться" + 0.089*"вид" + 0.073*"показывать" + 0.039*"окно" + 0.031*"такой" + 0.025*"сеть" + 0.023*"сказать" + 0.022*"отличаться" + 0.021*"ожидание" + 0.020*"мошенничество"
2017-10-12 16:51:32,617 : INFO : topic #40 (0.007): 0.200*"деньги" + 0.178*"верн

2017-10-12 16:51:32,680 : INFO : topic #72 (0.007): 0.177*"какой" + 0.127*"образ" + 0.103*"компания" + 0.084*"февраль" + 0.066*"подсказывать" + 0.059*"страхование" + 0.042*"жизнь" + 0.040*"страховой" + 0.021*"прошлое" + 0.020*"обязательный"
2017-10-12 16:51:32,682 : INFO : topic #73 (0.007): 0.126*"московский" + 0.125*"посмотреть" + 0.077*"декабрь" + 0.065*"акция" + 0.061*"подарок" + 0.052*"банк" + 0.051*"выплачивать" + 0.050*"область" + 0.044*"забросить" + 0.034*"промокод"
2017-10-12 16:51:32,684 : INFO : topic #74 (0.007): 0.202*"закрывать" + 0.125*"счет" + 0.058*"ожидать" + 0.048*"называться" + 0.047*"вклад" + 0.046*"момент" + 0.045*"процент" + 0.045*"орывать" + 0.038*"сказать" + 0.032*"перечисление"
2017-10-12 16:51:32,686 : INFO : topic #75 (0.007): 0.198*"код" + 0.104*"банкегательный" + 0.076*"банк" + 0.075*"дополнительный" + 0.064*"получать" + 0.050*"действовать" + 0.048*"промой" + 0.047*"учет" + 0.042*"пин" + 0.039*"тыср"
2017-10-12 16:51:32,687 : INFO : topic #76 (0.007): 0.14

2017-10-12 16:51:32,737 : INFO : topic #107 (0.007): 0.149*"вкладка" + 0.130*"производить" + 0.073*"окончание" + 0.069*"подобный" + 0.042*"онлайн" + 0.032*"банк" + 0.029*"отражаться" + 0.028*"взламывать" + 0.027*"уверенный" + 0.024*"графа"
2017-10-12 16:51:32,739 : INFO : topic #108 (0.007): 0.204*"звонить" + 0.100*"горячий" + 0.098*"номер" + 0.084*"линия" + 0.060*"позвонить" + 0.046*"банк" + 0.040*"перезванивать" + 0.036*"говорить" + 0.031*"предлагать" + 0.022*"почему"
2017-10-12 16:51:32,741 : INFO : topic #109 (0.007): 0.170*"задавать" + 0.168*"вопрос" + 0.089*"вспомнить" + 0.057*"паспортный" + 0.031*"тп" + 0.027*"данные" + 0.023*"отвечать" + 0.021*"мочь" + 0.021*"написание" + 0.020*"екатеринбург"
2017-10-12 16:51:32,744 : INFO : topic #110 (0.007): 0.199*"процент" + 0.086*"карта" + 0.061*"платить" + 0.059*"сумма" + 0.052*"начисляться" + 0.044*"пользоваться" + 0.040*"вопрос" + 0.039*"кредитный" + 0.036*"быть" + 0.036*"заплатить"
2017-10-12 16:51:32,746 : INFO : topic #111 (0.007): 0

2017-10-12 16:51:32,806 : INFO : topic #143 (0.007): 0.117*"банк" + 0.089*"повод" + 0.080*"наличие" + 0.079*"талант" + 0.061*"обращаться" + 0.045*"ноябрь" + 0.042*"прохождение" + 0.041*"практик" + 0.040*"скажитепожалуйста" + 0.036*"мой"
2017-10-12 16:51:32,807 : INFO : topic #144 (0.007): 0.354*"иметь" + 0.087*"право" + 0.078*"приход" + 0.073*"банкомат" + 0.067*"шишкин" + 0.056*"софья" + 0.046*"время" + 0.034*"банк" + 0.023*"высоко" + 0.014*"государственный"
2017-10-12 16:51:32,808 : INFO : topic #145 (0.007): 0.136*"зарплата" + 0.121*"карта" + 0.107*"получать" + 0.085*"выдача" + 0.052*"другой" + 0.049*"банкх" + 0.047*"данный" + 0.043*"момент" + 0.042*"мочь" + 0.036*"подходить"
2017-10-12 16:51:32,810 : INFO : topic #146 (0.007): 0.197*"ситуация" + 0.133*"такой" + 0.106*"случай" + 0.080*"факт" + 0.079*"данна" + 0.073*"пао" + 0.046*"держатель" + 0.027*"цифра" + 0.020*"банк" + 0.019*"местный"
2017-10-12 16:51:32,812 : INFO : topic #147 (0.007): 0.141*"оставаться" + 0.117*"счта" + 0.107*"

[(0,
  '0.189*"личный" + 0.158*"пароль" + 0.149*"кабинет" + 0.092*"банк" + 0.073*"зайти" + 0.067*"онлайн" + 0.050*"мочь" + 0.022*"заходить" + 0.021*"писать" + 0.019*"логин"'),
 (1,
  '0.144*"адрес" + 0.115*"заполнять" + 0.085*"проверка" + 0.080*"мир" + 0.064*"верить" + 0.060*"электронный" + 0.053*"ульяновск" + 0.041*"почта" + 0.031*"анкета" + 0.028*"банк"'),
 (2,
  '0.110*"дата" + 0.103*"оператор" + 0.094*"хотеться" + 0.084*"утро" + 0.076*"добрый" + 0.049*"изменяться" + 0.043*"узнавать" + 0.033*"связь" + 0.028*"помнить" + 0.028*"верный"'),
 (3,
  '0.321*"один" + 0.117*"регистрация" + 0.114*"разный" + 0.079*"выпускать" + 0.077*"два" + 0.070*"банк" + 0.057*"другой" + 0.043*"виртуальный" + 0.032*"карта" + 0.018*"онлайн"'),
 (4,
  '0.237*"имя" + 0.087*"посчитать" + 0.048*"самостоятельно" + 0.045*"потеря" + 0.039*"пользователь" + 0.037*"поручитель" + 0.032*"мочь" + 0.027*"мой" + 0.026*"версия" + 0.022*"поехать"'),
 (5,
  '0.142*"паспорт" + 0.141*"старый" + 0.134*"место" + 0.070*"прописка" +

In [338]:
index = similarities.MatrixSimilarity(lda[corpus])


2017-10-12 16:51:45,592 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2017-10-12 16:53:03,552 : INFO : creating matrix with 40117 documents and 150 features


In [361]:
def utterance_to_bow(utterance):
    utterance=utterance.lower()
    utterance=utterance.replace("тк","").replace("сбербанк","банк").replace("сбер","банк").replace("сбербанка","банк").replace("банка","банк")
    utterance=re.sub(r'[^а-яА-Я ]+', '',utterance)
    tokens = [word for word in str(utterance).split() if word not in stoplist]
    import pymystem3
    mystem = pymystem3.Mystem()
    utterance = [mystem.lemmatize(token)[0] for token in tokens]
    bow=dictionary.doc2bow(utterance)
    return bow

def utterance_to_result(utterance):
    vec_lda = lda[utterance_to_bow(utterance)]
    sims = index[vec_lda]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])[0:5]
    print(df['answer'][sims[0][0]])
    return sims
    

In [377]:
#"  вы можете, использовать кредитую карту visa/mastercard cre..
print(utterance_to_result("кредитная карта моментум. ею можно расплачиваться в зарубежных интернет магазинах?"))
#test
print(utterance_to_result("можно ли расплатиться кредитной картой моментумом в заграничных интернет магазинах"))
#test
print(utterance_to_result("мне нужно оплатить покупку в интернет магазине кредитной картой моментум"))

  вы можете, использовать кредитую карту visa/mastercard credit momentum для оплаты покупок в интернете при условии, что сайт поддерживает обслуживание данного вида карт.
[(0, 0.99998349), (32484, 0.80798149), (17350, 0.79935831), (22021, 0.76979363), (28727, 0.7382279)]
  вы можете, использовать кредитую карту visa/mastercard credit momentum для оплаты покупок в интернете при условии, что сайт поддерживает обслуживание данного вида карт.
[(0, 0.87707901), (18611, 0.77584082), (3510, 0.77328706), (3516, 0.7732538), (32484, 0.76566178)]
 все карты международных платежных систем visa international, mastercard international и american express, выпущенные сбербанком россии, можно использовать для оплаты покупок в интернете, кроме карт сбербанк-maestro «социальная».
[(19673, 0.95424402), (5871, 0.9142046), (12967, 0.9117623), (19530, 0.9117623), (21799, 0.9117623)]


In [379]:
#ля получения информации о лимите доступных средств по 
print(utterance_to_result("здоавствуйте, у меня подключен мобильный банк. как можно не выходя из дома проверить остаток на карточке?"))

#test
print(utterance_to_result("могу ли я через мобильный банк проверить остаток карты из дома"))


  для получения информации о лимите доступных средств по карте необходимо сформировать и направить на специальный номер оператора мобильной связи 900 или +7(926)200-09-00 или +7(916)572-39-00 с номера телефона, который подключен к услуге «мобильный банк», sms-сообщение следующего формата:  баланс 1234  где 1234 – последние 4 цифры номера вашей карты.  вместо баланс в sms-сообщении могут быть указаны следующие слова: остаток, ostatok, balans, balance, 01.
[(129, 0.99999994), (1840, 0.88764387), (4620, 0.88759601), (12755, 0.88759434), (39024, 0.8763538)]
  для получения информации о лимите доступных средств по карте необходимо сформировать и направить на специальный номер оператора мобильной связи 900 или +7(926)200-09-00 или +7(916)572-39-00 с номера телефона, который подключен к услуге «мобильный банк», sms-сообщение следующего формата:  баланс 1234  где 1234 – последние 4 цифры номера вашей карты.  вместо баланс в sms-сообщении могут быть указаны следующие слова: остаток, ostatok, ba

In [380]:
# вадим, в случае утраты карты во избежание ф
print(utterance_to_result("что делать если потерял карточку????  я потерял карточку правда на ней денег  уже не было,  но она мне все равно нужна чтобы потом деньги на неё класть, как её востановить? прийти в банк, и что дальше?"))
#test
print(utterance_to_result("я потерял карточку как ее восстановить"))

  вадим, в случае утраты карты во избежание финансовых потерь необходимо незамедлительно приостановить ее действие. заблокировать карту можно:  • через мобильный банк.  • через сбербанк онл@йн.  • через систему автоматизированного обслуживания.  • позвонив в службу помощи сбербанка по круглосуточным телефонам 8-800-555-55-50 или 8-800-200-37-47.  уточнить информацию о ваших дальнейших действиях более подробно вы можете, обратившись в службу помощи сбербанка.
[(132, 0.99999994), (37257, 0.68295085), (8602, 0.65515351), (34861, 0.64688855), (24892, 0.63671708)]
 досрочный перевыпуск карты может быть осуществлен в любом отделении банка и в контактном центре по телефону 8-800-555-555-0. основанием для досрочного перевыпуска может быть утрата карты, ее техническая неисправность, утрата пин-кода или смена именных данных. также перевыпуск карты осуществляется в связи с компрометацией данных карты.
[(18025, 0.98763359), (37257, 0.90518391), (2505, 0.75631738), (35788, 0.75151449), (20294, 0.70

In [383]:
#если вам было отказано в получении кредита в нашем банке, то повторно подать за
print(utterance_to_result("через какое время можно подать повторную заявку на кредит ?"))
#test
print(utterance_to_result("я подала заявку на кредит наличными мне отказали когда я смогу подать новую"))

 если вам было отказано в получении кредита в нашем банке, то повторно подать заявку вы можете по истечении 60 дней с момента получения отказа.
[(2732, 1.0), (7333, 1.0), (8039, 1.0), (12023, 1.0), (32047, 0.97936904)]
 в случае отказа вы можете повторно обратиться в банк с заявлением через 60 календарных дней.
[(28800, 0.87672496), (34528, 0.87378055), (14945, 0.86599386), (18859, 0.86466074), (37053, 0.85943258)]
